### Carregamento

In [5]:
# Importações necessárias
import pandas as pd

# Função para carregar os dados
def carregar_dados(caminho_arquivo, coluna_data=None):
    """
    Carrega um arquivo CSV com dados.
    
    Parâmetros:
    - caminho_arquivo (str): Caminho do arquivo CSV.
    - coluna_data (str, opcional): Nome da coluna de datas para converter para datetime.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os dados carregados.
    """
    # Carrega o CSV
    dados = pd.read_csv(caminho_arquivo)
    
    # Converte a coluna de datas para datetime com timezone, se especificada
    if coluna_data:
        dados[coluna_data] = pd.to_datetime(dados[coluna_data], utc=True)
    
    return dados

# Exemplo de uso da função com coluna de datas 'date'
caminho_arquivo = '../data/raw/train.csv'
dados = carregar_dados(caminho_arquivo)

# Exibir as primeiras linhas do DataFrame
dados.head(10)

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0
5,58896,Analytics,region_2,Bachelor's,m,sourcing,2,31,3.0,7,0,0,85,0
6,20379,Operations,region_20,Bachelor's,f,other,1,31,3.0,5,0,0,59,0
7,16290,Operations,region_34,Master's & above,m,sourcing,1,33,3.0,6,0,0,63,0
8,73202,Analytics,region_20,Bachelor's,m,other,1,28,4.0,5,0,0,83,0
9,28911,Sales & Marketing,region_1,Master's & above,m,sourcing,1,32,5.0,5,1,0,54,0


### Tamanho da base

In [6]:
def verificar_tamanho_base(df):
    """
    Verifica o tamanho da base de dados em termos de número de linhas e colunas.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    
    Retorna:
    - dict: Dicionário com o número de linhas e colunas.
    """
    tamanho = {
        'Número de Linhas': df.shape[0],
        'Número de Colunas': df.shape[1]
    }
    
    return tamanho

# Verificar o tamanho da base de dados
tamanho_base = verificar_tamanho_base(dados)
tamanho_base

{'Número de Linhas': 54808, 'Número de Colunas': 14}

### Valores nulos

In [7]:
# Função para verificar valores ausentes
def verificar_valores_ausentes(df):
    """
    Verifica a quantidade e o percentual de valores ausentes em cada coluna de um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    
    Retorna:
    - pandas.DataFrame: DataFrame com a quantidade e o percentual de valores ausentes por coluna.
    """
    # Calcular a quantidade de valores ausentes
    valores_ausentes = df.isnull().sum()
    
    # Calcular o percentual de valores ausentes
    percentual_ausentes = (valores_ausentes / df.shape[0]) * 100
    
    # Criar um DataFrame com os resultados
    tabela_ausentes = pd.DataFrame({
        'Quantidade de Valores Ausentes': valores_ausentes,
        'Percentual de Valores Ausentes (%)': percentual_ausentes
    })
    
    # Ordenar o DataFrame pelo percentual de valores ausentes em ordem decrescente
    tabela_ausentes = tabela_ausentes[tabela_ausentes['Quantidade de Valores Ausentes'] > 0]
    tabela_ausentes = tabela_ausentes.sort_values(by='Percentual de Valores Ausentes (%)', ascending=False)
    
    return tabela_ausentes

# Verificar valores ausentes
tabela_ausentes = verificar_valores_ausentes(dados)
tabela_ausentes

,Quantidade de Valores Ausentes,Percentual de Valores Ausentes (%)
previous_year_rating,4124,7.524449
education,2409,4.395344


### Valores nulos: previous_year_rating

In [8]:
def substituir_valores_nulos_com_media(df, coluna):
    """
    Substitui os valores nulos de uma coluna pela média arredondada dessa coluna.

    Parâmetros:
    - df (pandas.DataFrame): DataFrame contendo a coluna.
    - coluna (str): Nome da coluna cujos valores nulos serão substituídos.

    Retorna:
    - pandas.DataFrame: DataFrame com os valores nulos substituídos.
    - str: Mensagem indicando o sucesso da operação e o número de valores nulos substituídos.
    """
    # Calcular a média da coluna, ignorando os valores nulos
    media = df[coluna].mean()
    
    # Arredondar a média para o inteiro mais próximo
    media_arredondada = round(media)
    
    # Contar o número de valores nulos antes da substituição
    nulos_antes = df[coluna].isnull().sum()
    
    # Substituir os valores nulos pela média arredondada
    df[coluna] = df[coluna].fillna(media_arredondada)
    
    # Contar o número de valores nulos após a substituição
    nulos_depois = df[coluna].isnull().sum()
    
    # Verificar se a operação foi um sucesso
    if nulos_depois == 0:
        mensagem = f"Operação bem-sucedida! {nulos_antes} valores nulos foram substituídos na coluna '{coluna}'."
    else:
        mensagem = f"Operação parcialmente bem-sucedida. Restam {nulos_depois} valores nulos na coluna '{coluna}'."
    
    return df, mensagem

# Aplicar a função ao DataFrame 'dados' na coluna 'previous_year_rating'
dados, mensagem = substituir_valores_nulos_com_media(dados, 'previous_year_rating')

# Exibir a mensagem
print(mensagem)

# Verificar se os valores nulos foram substituídos
dados['previous_year_rating'].isnull().sum()

Operação bem-sucedida! 4124 valores nulos foram substituídos na coluna 'previous_year_rating'.


0

### Conferência substituição.

In [9]:
# Função para verificar valores ausentes
def verificar_valores_ausentes(df):
    """
    Verifica a quantidade e o percentual de valores ausentes em cada coluna de um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    
    Retorna:
    - pandas.DataFrame: DataFrame com a quantidade e o percentual de valores ausentes por coluna.
    """
    # Calcular a quantidade de valores ausentes
    valores_ausentes = df.isnull().sum()
    
    # Calcular o percentual de valores ausentes
    percentual_ausentes = (valores_ausentes / df.shape[0]) * 100
    
    # Criar um DataFrame com os resultados
    tabela_ausentes = pd.DataFrame({
        'Quantidade de Valores Ausentes': valores_ausentes,
        'Percentual de Valores Ausentes (%)': percentual_ausentes
    })
    
    # Ordenar o DataFrame pelo percentual de valores ausentes em ordem decrescente
    tabela_ausentes = tabela_ausentes[tabela_ausentes['Quantidade de Valores Ausentes'] > 0]
    tabela_ausentes = tabela_ausentes.sort_values(by='Percentual de Valores Ausentes (%)', ascending=False)
    
    return tabela_ausentes

# Verificar valores ausentes
tabela_ausentes = verificar_valores_ausentes(dados)
tabela_ausentes

,Quantidade de Valores Ausentes,Percentual de Valores Ausentes (%)
education,2409,4.395344


### Valores nulos: education

In [10]:
def substituir_valores_nulos_proporcional(df, coluna):
    """
    Substitui os valores nulos de uma coluna por valores únicos de forma proporcional à sua frequência.

    Parâmetros:
    - df (pandas.DataFrame): DataFrame contendo a coluna.
    - coluna (str): Nome da coluna cujos valores nulos serão substituídos.

    Retorna:
    - pandas.DataFrame: DataFrame com os valores nulos substituídos.
    - str: Mensagem indicando o sucesso da operação e a distribuição dos valores antes e depois da substituição.
    """
    # Contar a frequência de cada valor único na coluna
    frequencia = df[coluna].value_counts()
    
    # Calcular a proporção de cada valor único
    proporcao = frequencia / frequencia.sum()
    
    # Contar o número de valores nulos na coluna
    nulos_antes = df[coluna].isnull().sum()
    
    # Calcular o número de valores nulos a serem substituídos por cada valor único
    substituicoes = (proporcao * nulos_antes).round().astype(int)
    
    # Criar uma lista com os valores a serem usados para substituir os nulos
    valores_substituicao = []
    for valor, quantidade in substituicoes.items():
        valores_substituicao.extend([valor] * quantidade)
    
    # Verificar se a soma das substituições é igual ao número de nulos
    if len(valores_substituicao) < nulos_antes:
        valores_substituicao.extend([frequencia.idxmax()] * (nulos_antes - len(valores_substituicao)))
    elif len(valores_substituicao) > nulos_antes:
        valores_substituicao = valores_substituicao[:nulos_antes]
    
    # Substituir os valores nulos na coluna
    df.loc[df[coluna].isnull(), coluna] = valores_substituicao
    
    # Contar a frequência de cada valor único após a substituição
    frequencia_apos = df[coluna].value_counts()
    
    # Criar a mensagem de saída
    mensagem = f"Operação bem-sucedida! {nulos_antes} valores nulos foram substituídos na coluna '{coluna}'.\n"
    mensagem += "Distribuição dos valores antes e depois da substituição:\n"
    for valor in frequencia.index:
        mensagem += f" - {valor}: {frequencia[valor]} -> {frequencia_apos[valor]}\n"
    
    return df, mensagem

# Aplicar a função ao DataFrame 'dados' na coluna 'education'
dados, mensagem = substituir_valores_nulos_proporcional(dados, 'education')

# Exibir a mensagem
print(mensagem)

# Verificar se os valores nulos foram substituídos
dados['education'].isnull().sum()

Operação bem-sucedida! 2409 valores nulos foram substituídos na coluna 'education'.
Distribuição dos valores antes e depois da substituição:
 - Bachelor's: 36669 -> 38355
 - Master's & above: 14925 -> 15611
 - Below Secondary: 805 -> 842



0

### Conferência substituição

In [11]:
# Função para verificar valores ausentes
def verificar_valores_ausentes(df):
    """
    Verifica a quantidade e o percentual de valores ausentes em cada coluna de um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    
    Retorna:
    - pandas.DataFrame: DataFrame com a quantidade e o percentual de valores ausentes por coluna.
    """
    # Calcular a quantidade de valores ausentes
    valores_ausentes = df.isnull().sum()
    
    # Calcular o percentual de valores ausentes
    percentual_ausentes = (valores_ausentes / df.shape[0]) * 100
    
    # Criar um DataFrame com os resultados
    tabela_ausentes = pd.DataFrame({
        'Quantidade de Valores Ausentes': valores_ausentes,
        'Percentual de Valores Ausentes (%)': percentual_ausentes
    })
    
    # Ordenar o DataFrame pelo percentual de valores ausentes em ordem decrescente
    tabela_ausentes = tabela_ausentes[tabela_ausentes['Quantidade de Valores Ausentes'] > 0]
    tabela_ausentes = tabela_ausentes.sort_values(by='Percentual de Valores Ausentes (%)', ascending=False)
    
    return tabela_ausentes

# Verificar valores ausentes
tabela_ausentes = verificar_valores_ausentes(dados)
tabela_ausentes

,Quantidade de Valores Ausentes,Percentual de Valores Ausentes (%)


### Valores duplicados

In [12]:
def detectar_valores_duplicados(df):
    """
    Detecta e remove valores duplicados de um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado
    
    Retorna:
    - pandas.DataFrame: DataFrame sem duplicatas
    """
    # Verifica duplicatas
    duplicatas = df.duplicated(keep=False)
    n_duplicatas = duplicatas.sum()
    
    if n_duplicatas == 0:
        print("✅ Nenhum valor duplicado foi encontrado.")
        return df
    else:
        print(f"⚠️ Foram encontradas {n_duplicatas} linhas duplicadas.")
        # Remove duplicatas mantendo a primeira ocorrência
        df_sem_duplicatas = df.drop_duplicates(keep='first')
        return df_sem_duplicatas

# Aplicar a função e atualizar o DataFrame
dados = detectar_valores_duplicados(dados)
dados.head()

✅ Nenhum valor duplicado foi encontrado.


,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


### Verificação tipos dos dados

In [13]:
# Função para verificar o tipo dos dados/colunas
def verificar_tipo_dados(df):
    """
    Verifica o tipo dos dados/colunas de um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os nomes das colunas e seus respectivos tipos de dados.
    """
    # Obter os tipos de dados das colunas
    tipos_dados = df.dtypes
    
    # Criar um DataFrame com os resultados
    tabela_tipos = pd.DataFrame({
        'Coluna': tipos_dados.index,
        'Tipo de Dado': tipos_dados.values
    })
    
    return tabela_tipos

# Verificar o tipo dos dados/colunas
tabela_tipos = verificar_tipo_dados(dados)
tabela_tipos

,Coluna,Tipo de Dado
0,employee_id,int64
1,department,object
2,region,object
3,education,object
4,gender,object
5,recruitment_channel,object
6,no_of_trainings,int64
7,age,int64
8,previous_year_rating,float64
9,length_of_service,int64


### Conversão/otimização tipos

In [17]:
def modificar_tipo_colunas(df, colunas_tipos):
    """
    Modifica o tipo de dado de colunas específicas em um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame original.
    - colunas_tipos (dict): Dicionário onde as chaves são os nomes das colunas e os valores são os tipos de dados desejados.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os tipos de dados modificados.
    """
    # Verifica se as colunas informadas existem no DataFrame
    colunas_invalidas = [col for col in colunas_tipos.keys() if col not in df.columns]
    if colunas_invalidas:
        raise ValueError(f"Colunas inválidas: {colunas_invalidas}")
    
    # Modifica o tipo de dado das colunas especificadas
    for coluna, tipo in colunas_tipos.items():
        try:
            df[coluna] = df[coluna].astype(tipo)
            print(f"✅ Coluna '{coluna}' convertida para '{tipo}' com sucesso.")
        except Exception as e:
            print(f"⚠️ Erro ao converter a coluna '{coluna}' para '{tipo}': {e}")
    
    return df

# Exemplo de uso:
colunas_tipos = {
    'employee_id': 'int32',
    'department': 'category',
    'region': 'category',
    'education': 'category',
    'gender': 'category',
    'recruitment_channel': 'category',
    'no_of_trainings': 'int8',
    'age': 'int8',
    'previous_year_rating': 'float8',
    'length_of_service': 'int8',
    'KPIs_met >80%': 'int8',
    'awards_won?': 'int8',
    'avg_training_score': 'int8',
    'is_promoted': 'int8'
    }

dados = modificar_tipo_colunas(dados, colunas_tipos)
dados.dtypes # Verificar os tipos de dados


✅ Coluna 'employee_id' convertida para 'int32' com sucesso.
✅ Coluna 'department' convertida para 'category' com sucesso.
✅ Coluna 'region' convertida para 'category' com sucesso.
✅ Coluna 'education' convertida para 'category' com sucesso.
✅ Coluna 'gender' convertida para 'category' com sucesso.
✅ Coluna 'recruitment_channel' convertida para 'category' com sucesso.
✅ Coluna 'no_of_trainings' convertida para 'int8' com sucesso.
✅ Coluna 'age' convertida para 'int8' com sucesso.
⚠️ Erro ao converter a coluna 'previous_year_rating' para 'float8': data type 'float8' not understood
✅ Coluna 'length_of_service' convertida para 'int8' com sucesso.
✅ Coluna 'KPIs_met >80%' convertida para 'int8' com sucesso.
✅ Coluna 'awards_won?' convertida para 'int8' com sucesso.
✅ Coluna 'avg_training_score' convertida para 'int8' com sucesso.
✅ Coluna 'is_promoted' convertida para 'int8' com sucesso.


employee_id                int32
department              category
region                  category
education               category
gender                  category
recruitment_channel     category
no_of_trainings             int8
age                         int8
previous_year_rating     float32
length_of_service           int8
KPIs_met >80%               int8
awards_won?                 int8
avg_training_score          int8
is_promoted                 int8
dtype: object

In [18]:
def modificar_tipo_colunas(df, colunas_tipos):
    """
    Modifica o tipo de dado de colunas específicas em um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame original.
    - colunas_tipos (dict): Dicionário onde as chaves são os nomes das colunas e os valores são os tipos de dados desejados.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os tipos de dados modificados.
    """
    # Verifica se as colunas informadas existem no DataFrame
    colunas_invalidas = [col for col in colunas_tipos.keys() if col not in df.columns]
    if colunas_invalidas:
        raise ValueError(f"Colunas inválidas: {colunas_invalidas}")
    
    # Modifica o tipo de dado das colunas especificadas
    for coluna, tipo in colunas_tipos.items():
        try:
            df[coluna] = df[coluna].astype(tipo)
            print(f"✅ Coluna '{coluna}' convertida para '{tipo}' com sucesso.")
        except Exception as e:
            print(f"⚠️ Erro ao converter a coluna '{coluna}' para '{tipo}': {e}")
    
    return df

# Exemplo de uso:
colunas_tipos = {
    'previous_year_rating': 'float32',
    }

dados = modificar_tipo_colunas(dados, colunas_tipos)
dados.dtypes # Verificar os tipos de dados


✅ Coluna 'previous_year_rating' convertida para 'float32' com sucesso.


employee_id                int32
department              category
region                  category
education               category
gender                  category
recruitment_channel     category
no_of_trainings             int8
age                         int8
previous_year_rating     float32
length_of_service           int8
KPIs_met >80%               int8
awards_won?                 int8
avg_training_score          int8
is_promoted                 int8
dtype: object

### Verificação dos dados númericos

In [19]:
def verificar_colunas_numericas(df, colunas):
    """
    Verifica se as colunas especificadas contêm apenas valores numéricos.

    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    - colunas (list): Lista de nomes das colunas a serem verificadas.

    Retorna:
    - dict: Dicionário com o nome da coluna e um booleano indicando se contém apenas números.
    - pandas.DataFrame: DataFrame com as inconsistências encontradas.
    """
    resultado = {}
    inconsistencias = []

    for coluna in colunas:
        if coluna in df.columns:
            # Verifica se todos os valores na coluna são numéricos
            col_numerica = pd.to_numeric(df[coluna], errors='coerce')
            resultado[coluna] = col_numerica.notnull().all()
            if not resultado[coluna]:
                # Adiciona as inconsistências à lista
                posicoes_inconsistentes = col_numerica[col_numerica.isnull()].index.tolist()
                for posicao in posicoes_inconsistentes:
                    inconsistencias.append({'Coluna': coluna, 'Posição': posicao, 'Valor': df.at[posicao, coluna]})
        else:
            resultado[coluna] = False
            print(f"⚠️ Coluna '{coluna}' não encontrada no DataFrame.")

    # Cria um DataFrame com as inconsistências
    df_inconsistencias = pd.DataFrame(inconsistencias)

    return resultado, df_inconsistencias

# Colunas a serem verificadas
colunas_a_verificar = [
    'employee_id', 'no_of_trainings', 'age', 'previous_year_rating', 
    'length_of_service', 'KPIs_met >80%', 'awards_won?', 
    'avg_training_score', 'is_promoted'
]

# Verificar as colunas no DataFrame 'dados'
resultado_verificacao, df_inconsistencias = verificar_colunas_numericas(dados, colunas_a_verificar)

# Exibir o resultado da verificação
for coluna, is_numerica in resultado_verificacao.items():
    status = "contém apenas números" if is_numerica else "não contém apenas números"
    print(f"Coluna '{coluna}': {status}")

# Exibir as inconsistências, se houver
if not df_inconsistencias.empty:
    print("\nInconsistências encontradas:")
    display(df_inconsistencias)
else:
    print("\nNenhuma inconsistência encontrada nas colunas verificadas.")

Coluna 'employee_id': contém apenas números
Coluna 'no_of_trainings': contém apenas números
Coluna 'age': contém apenas números
Coluna 'previous_year_rating': contém apenas números
Coluna 'length_of_service': contém apenas números
Coluna 'KPIs_met >80%': contém apenas números
Coluna 'awards_won?': contém apenas números
Coluna 'avg_training_score': contém apenas números
Coluna 'is_promoted': contém apenas números

Nenhuma inconsistência encontrada nas colunas verificadas.


In [22]:
def detectar_valores_unicos(df, coluna):
    """
    Detecta valores únicos em uma coluna de um DataFrame e exibe esses valores de forma amigável.

    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser analisado.
    - coluna (str): Nome da coluna cujos valores únicos serão detectados.

    Retorna:
    - pandas.DataFrame: DataFrame com os valores únicos e suas contagens.
    - str: Mensagem indicando o sucesso da operação e o número de valores únicos encontrados.
    """
    if coluna not in df.columns:
        raise ValueError(f"Coluna '{coluna}' não encontrada no DataFrame.")
    
    # Detectar valores únicos e suas contagens
    valores_unicos = df[coluna].value_counts().reset_index()
    valores_unicos.columns = [coluna, 'Contagem']
    
    # Criar a mensagem de saída
    mensagem = f"Operação bem-sucedida! Foram encontrados {valores_unicos.shape[0]} valores únicos na coluna '{coluna}'."
    
    return valores_unicos, mensagem

# Exemplo de uso:
colunas = ['department', 'region', 'education', 'gender', 'recruitment_channel']
for coluna in colunas:
    valores_unicos, mensagem = detectar_valores_unicos(dados, coluna)
    # Exibir a mensagem e a tabela de valores únicos
    print(mensagem)
    display(valores_unicos)

Operação bem-sucedida! Foram encontrados 9 valores únicos na coluna 'department'.


,department,Contagem
0,Sales & Marketing,16840
1,Operations,11348
2,Procurement,7138
3,Technology,7138
4,Analytics,5352
5,Finance,2536
6,HR,2418
7,Legal,1039
8,R&D,999


Operação bem-sucedida! Foram encontrados 34 valores únicos na coluna 'region'.


,region,Contagem
0,region_2,12343
1,region_22,6428
2,region_7,4843
3,region_15,2808
4,region_13,2648
5,region_26,2260
6,region_31,1935
7,region_4,1703
8,region_27,1659
9,region_16,1465


Operação bem-sucedida! Foram encontrados 3 valores únicos na coluna 'education'.


,education,Contagem
0,Bachelor's,38355
1,Master's & above,15611
2,Below Secondary,842


Operação bem-sucedida! Foram encontrados 2 valores únicos na coluna 'gender'.


,gender,Contagem
0,m,38496
1,f,16312


Operação bem-sucedida! Foram encontrados 3 valores únicos na coluna 'recruitment_channel'.


,recruitment_channel,Contagem
0,other,30446
1,sourcing,23220
2,referred,1142


### Remover possíveis espaços

In [23]:
def remover_espacos(df):
    """
    Remove espaços em branco à esquerda e à direita das strings em todas as colunas de um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser processado.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os espaços removidos.
    - pandas.DataFrame: DataFrame com informações sobre as remoções realizadas.
    """
    # Verificar se o DataFrame é válido
    if not isinstance(df, pd.DataFrame):
        raise ValueError("O argumento fornecido não é um DataFrame.")
    
    # Inicializar lista para armazenar informações sobre remoções
    informacoes_remocoes = []

    # Aplicar strip em todas as colunas de tipo object e category
    for col in df.select_dtypes(include=['object', 'category']).columns:
        # Contar espaços em branco antes da remoção
        espacos_antes = df[col].apply(lambda x: x != x.strip() if isinstance(x, str) else False).sum()
        
        # Remover espaços em branco
        df[col] = df[col].map(lambda x: x.strip() if isinstance(x, str) else x)
        
        # Contar espaços em branco após a remoção
        espacos_depois = df[col].apply(lambda x: x != x.strip() if isinstance(x, str) else False).sum()
        
        # Verificar se a remoção foi bem-sucedida
        sucesso = espacos_depois == 0
        
        # Adicionar informações à lista
        informacoes_remocoes.append({
            'Coluna': col,
            'Espaços Antes': espacos_antes,
            'Espaços Depois': espacos_depois,
            'Remoção Bem-Sucedida': sucesso
        })
    
    # Criar DataFrame com as informações de remoção
    df_informacoes_remocoes = pd.DataFrame(informacoes_remocoes)
    
    return df, df_informacoes_remocoes

# Exemplo de uso:
dados, informacoes_remocoes = remover_espacos(dados)

# Exibir as informações sobre as remoções
print("Informações sobre a remoção de espaços em branco:")
display(informacoes_remocoes)

Informações sobre a remoção de espaços em branco:


,Coluna,Espaços Antes,Espaços Depois,Remoção Bem-Sucedida
0,department,0,0,True
1,region,0,0,True
2,education,0,0,True
3,gender,0,0,True
4,recruitment_channel,0,0,True


### Carga

In [24]:
def salvar_dados(df, caminho_arquivo_saida):
    """
    Salva o DataFrame em um arquivo CSV.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser salvo.
    - caminho_arquivo_saida (str): Caminho do arquivo CSV de saída.
    """
    df.to_csv(caminho_arquivo_saida, index=False)
    print(f"✅ Dados salvos com sucesso em '{caminho_arquivo_saida}'")

# Exemplo de uso:
caminho_arquivo_saida = '../data/processed/train_atualizado.csv'
salvar_dados(dados, caminho_arquivo_saida)

✅ Dados salvos com sucesso em '../data/processed/train_atualizado.csv'
